# Modelo Preditivo de Fraude
Random Forest sem Cross validation

## Objetivo
Os POCs reportados são focados em agências com maior ocorrência de fraude, não constituindo, portanto, um registro completo dos POCs ocorridos. Esta análise tem por objetivo  a melhorar a performance do modelo preditivo, a partir da marcação dos POCs de forma mais completa, por meio da análise dos eventos ocorridos nas agências/ATMs com POCs reportados.

## Libraries

In [1]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)
from pyspark.sql.types import Row
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import atexit
from numpy import array
import numpy as np
import pandas as pd
from pyspark.mllib.linalg import Vectors
from pylab import plot,show,hist,figure,title
import unicodedata
from datetime import datetime, time
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# para plots
%matplotlib inline

## constantes

In [2]:
mediana = 50  # mediana da ocorrência de eventos diários (feature)
percentil_max = 95 # percentil máximo considerado para treinar o modelo
D_90 = 90 # 90 dias a subtrair (usado somente para treino inicial)
D_1 = 1 # obtem eventos de D-1 para previsão de POC em produção
D_2 = 2 # obtem eventos a aprtir de D-2 para trás para treino constante do modelo em produção

## Carga de Dados Brutos Hadoop Produção

In [ ]:
####################################################################################
# PARA RODAR EM HADOOP PRODUCAO DESCOMENTAR ABAIXO, COMENTANDO A CELULA DO MAC OSX 
####################################################################################
funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')\
                           .replace(minute=0, hour=0, second=0, microsecond=0), DateType())
funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

In [4]:
rdd_eventlog = hive_context.table('default.ext_eventos_fullog1908')
#poc_in = '/user/x181472/export/BASE_POC_ATM_090816.csv'
poc_in = '/user/x181472/export/V10_BASE_POC_ATM_060916.csv'

tranlog_in = hive_context.table('default.ext_juntatudo12')
tabelao_out = '/temporario/tabelao-temp-V5.2.5.csv'
tabelao_in = '/user/x184166/tabelao-temp-V5.2.5.csv' # Hermann
poceven_in = '/temporario/pokeventos-V2.csv'

In [5]:
df_eventlog = rdd_eventlog \
    .map(lambda p: (p[1],p[2],p[3], p[4])) \
    .toDF(['nr_sequ_even',
           'dh_even', 
           'cd_eqpt', 
           'nr_idef_secu' 
                   ])
df_eventlog.printSchema() 

print ("\n\nNumero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

# sumarizar a base por agencia+atm+evento+dia
df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

df_eventlog = df_eventlog\
    .groupBy(['nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'dh_even']).count()\
    .orderBy(col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even'), col('dh_even').desc())\
    .withColumnRenamed('count', 'nr_eventos')

root
 |-- nr_sequ_even: long (nullable = true)
 |-- dh_even: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_idef_secu: string (nullable = true)



Numero de linhas carregadas: 18341921


In [6]:
#df_eventlog.limit(10).toPandas()

## Carga de dados Brutos MAC

In [7]:
####################################################################################
# PARA RODAR MAC OSX DESCOMENTAR ABAIXO, COMENTANDO A CELULA DO HADOOP PRODUCAO 
####################################################################################
#eventlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tbfullog.csv'
#poc_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/BASE_POC_ATM_090816.csv'
#ranlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/dados/juntatudo11pordia-tranfin.csv'
#tabelao_out = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#tabelao_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#previsao = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/previsao-V5.2.2.csv'
#poceven_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/pokeventos.csv'

#funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
#funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

#rdd_eventlog = sc.textFile(eventlog_in)
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

#header = rdd_eventlog.first()

#df_eventlog = rdd_eventlog \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(",")) \
#   .map(lambda p: (p[1],p[2],p[3],p[4], p[5])) \
#    .toDF(['nr_sequ_even',
#            'dh_even', 
#            'cd_eqpt', 
#            'nr_idef_secu', 
#            'nr_eventos'
#                  ])
#df_eventlog.printSchema()    

In [8]:
#df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

In [9]:
df_eventlog = df_eventlog \
    .withColumn('nr_sequ_even', df_eventlog["nr_sequ_even"].cast(StringType()))\
    .withColumn('cd_eqpt', df_eventlog["cd_eqpt"].cast(StringType()))\
    .withColumn('nr_idef_secu', df_eventlog["nr_idef_secu"].cast(StringType())) \
    .withColumn('dh_even', df_eventlog["dh_even"].cast(TimestampType()))\
    .withColumn('nr_eventos', df_eventlog["nr_eventos"].cast(IntegerType()))

df_eventlog.printSchema()   

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)



In [10]:
#df_eventlog.limit(10).orderBy(col("nr_eventos").desc()).toPandas()

In [11]:
# DEPOIS: acertar abaixo para pegar direto ultima data do arquivo original como data_ref
# Filtrando últimos 90 dias
import datetime, time
from dateutil.relativedelta import relativedelta

# obtendo faixa do período de histórico a partir da data de referencia
data_ref = datetime.datetime(2016, 8, 10) 
D_menos1 = data_ref + relativedelta(days=-D_1)
D_menos2 = data_ref + relativedelta(days=-D_2)
D_menos90 = D_menos2 + relativedelta(days=-D_90)
print "D-90:   " + str(D_menos90)
print "D-1 :   " + str(D_menos1)
print "D-2 :   " + str(D_menos2)

D-90:   2016-05-10 00:00:00
D-1 :   2016-08-09 00:00:00
D-2 :   2016-08-08 00:00:00


In [12]:
dt_sql_90 = (time.mktime(datetime.datetime.strptime(str(D_menos90.date()), "%Y-%m-%d").timetuple()))
dt_sql_1 = (time.mktime(datetime.datetime.strptime(str(D_menos1.date()), "%Y-%m-%d").timetuple()))
dt_sql_2 = (time.mktime(datetime.datetime.strptime(str(D_menos2.date()), "%Y-%m-%d").timetuple()))
dt_sql_ref = (time.mktime(datetime.datetime.strptime(str(data_ref.date()), "%Y-%m-%d").timetuple()))
print dt_sql_90
print dt_sql_1
print dt_sql_2
print dt_sql_ref

1462849200.0
1470711600.0
1470625200.0
1470798000.0


In [13]:
# FILTRANDO SOMENTE ULTIMOS 90 DIAS, a partir de D-2
df_treinoevent = df_eventlog.where("CAST(dh_even AS INT) BETWEEN {0} AND {1}"\
                                   .format(dt_sql_90, dt_sql_2)).orderBy("dh_even")
df_treinoevent.limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos
0          105     164          632 2016-05-10           2
1          105     164         4557 2016-05-10           1
2          105     164          702 2016-05-10           1
3          105     163         6334 2016-05-10           1
4          105     164          723 2016-05-10           1
5          105     163         6732 2016-05-10           3
6          105     164          724 2016-05-10           1
7          105     163         7023 2016-05-10          18
8          105     164         3009 2016-05-10           2
9          105     164         3010 2016-05-10           1

In [14]:
#print ("\nNúmero de linhas carregadas: " + str("%d" % (df_treinoevent.count())))
# ABAIXO: COMPOPRTAMENTO ESTRANHO - traz mais linhas que o numero original apos dropna??!!
#print ("\nNúmero de linhas após retirar NAs: " + str("%d" % (df_treinoevent.dropna().count())))
#df_treinoevent = df_treinoevent.dropna()

In [15]:
#df_treinoevent.filter((col("nr_sequ_even") == '3321')).count()

In [16]:
#print ("\nNúmero de linhas carregadas: " + str("%d" % (df_treinoevent.count())))

## Obtendo Eventos Marcadores de POC (selecionados como features)
Carrega a lista de eventos marcadores de POC a partir de arquivo já gerado pelo notebook STDR-Seletor de Pokeventos.

In [17]:
# lendo arquivo de pokeventos e carregando em lista
pd_poceven = pd.read_csv(poceven_in, header=None).T
pd_poceven.columns = ['pokevento']

l_even = pd_poceven.pokevento.tolist()

def foo(l_even, dtype=str):
    return map(dtype, l_even)
l_evenpoc = foo(l_even)
l_features = l_evenpoc[0:50]

In [18]:
# pegar sample abaixo se a memoria nao suportar para trnaformar o Pandas em dataframe de pois de pivotar

#randomList = getRandom(l_features,100)
len(l_features)

50

In [19]:
# prepara lista de features para eventos se usar SQL ao inves de Pandas
#for i in range(len(l_features)):
#    l_features[i] = "ev" + str(l_features[i])

#l_features

In [20]:
# filtrando log para todos eventos considerados como features
df_treinoevent = df_treinoevent.filter((col("nr_sequ_even").isin(l_features)))

In [21]:
#df_treinoevent.limit(10).toPandas()

In [22]:
# calculando percentis de baseline para todas as observações do log com base em agencia/atm/evento
# OBS: o uso do mediana desta forma permite usar os eventos isoladamente como marcadores 
# pois normaliza-os, tornando este marcador independente das ATM onde ocorrem
df_eventlog_percen = df_treinoevent.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("cd_eqpt"), lit("-"),col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

In [23]:
#df_eventlog_percen.take(5)

In [24]:
rdd_eventlog_percen = df_eventlog_percen.rdd \
    .map(lambda x: (x[0], (x[1],), )) \
    .reduceByKey(lambda acc, val: acc + val) \
    .map(lambda x: (x[0], np.percentile(x[1], mediana), np.percentile(x[1], percentil_max)))

In [25]:
rdd_eventlog_percen = rdd_eventlog_percen \
    .map(lambda k: (k[0].split("-"), k[1], k[2])) 

In [26]:
df_eventlog_percen = rdd_eventlog_percen \
    .map(lambda p: (p[0][0],p[0][1],p[0][2],float(p[1]), float(p[2])  )) \
    .toDF(['nr_idef_secu',
           'cd_eqpt',
           'nr_sequ_even',
           'mediana',
           'percentil_max'
                   ])
#df_eventlog_percen.limit(10).toPandas()

In [27]:
df_eventlog_percen.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [28]:
#df_eventlog.count()

In [29]:
#df_eventlog.limit(10).toPandas()

In [30]:
# colocar no log total as colunas de mediana e percentil máximo por agencia+atm+evento 
df_eventlog2 = df_treinoevent \
    .join(df_eventlog_percen, (df_treinoevent['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (df_treinoevent['cd_eqpt'] == df_eventlog_percen['cd_eqpt']) &
                              (df_treinoevent['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
    .drop(df_eventlog_percen["nr_idef_secu"])\
    .drop(df_eventlog_percen["cd_eqpt"])\
    .drop(df_eventlog_percen["nr_sequ_even"])

In [31]:
#df_eventlog2.limit(10).toPandas()

In [32]:
#df_eventlog2.count()

In [33]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_eventlog2 = df_eventlog2\
    .withColumn('cd_eqpt', trim(df_eventlog2.cd_eqpt))\
    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda
#    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0', '')) # removendo zero `esquerda
# testar s.replaceFirst("^0+(?!$)", "")

In [34]:
#df_eventlog2.orderBy('nr_sequ_even').limit(10).toPandas()

### Dados de registro de POC
Obtém os POCs de Retenção de Cartão, somente para os últimos 90 dias.
Trata as seguintes colunas:
+ data de cadastro do POC
+ código da ATM onde ocorreu o POC
+ código da agência onde ocorreu o POC

Identifica quais eventos que aconteceram nas agências e na data que saíram da faixa de percentil. estes eventos serão os **Marcadores de POC**.

In [35]:
rdd_poc = sc.textFile(poc_in)
print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_poc.count())))

header = rdd_poc.first()
# pegando data inicial de comprometimento ao inves de data_cad, pois analise exploratoria mostrou que tem
# mais sentido usar esta data
df_poc = rdd_poc \
    .filter(lambda line: line != header) \
    .map(lambda k: k.split(";")) \
    .map(lambda p: (p[0], p[3], p[7])) \
    .toDF(['data_range', 
            'agencia',
            'golpe'
        ])
    
df_poc.toPandas()



Número de linhas carregadas: 903


data_range agencia                       golpe
0    09/05/2016    1546  GOLPE RETENTOR DE ENVELOPE
1    09/05/2016    1650         LEITORAS OBSTRUIDAS
2    09/05/2016     734         LEITORAS OBSTRUIDAS
3    09/05/2016    1546  GOLPE RETENTOR DE ENVELOPE
4    09/05/2016    4348  GOLPE RETENTOR DE ENVELOPE
5    09/04/2016     778              TELEFONE FALSO
6    09/04/2016     350  GOLPE RETENTOR DE ENVELOPE
7    09/04/2016    1206  GOLPE RETENTOR DE ENVELOPE
8    09/04/2016    1206  GOLPE RETENTOR DE ENVELOPE
9    09/03/2016     985  GOLPE RETENTOR DE ENVELOPE
10   09/03/2016     564  GOLPE RETENTOR DE ENVELOPE
11   09/03/2016    3793  GOLPE RETENTOR DE ENVELOPE
12   09/03/2016     768  GOLPE RETENTOR DE ENVELOPE
13   09/03/2016    4442    GOLPE RETENTOR DE CARTAO
14   09/03/2016     921  GOLPE RETENTOR DE ENVELOPE
15   09/03/2016     919  GOLPE RETENTOR DE ENVELOPE
16    8/30/2016    1566  GOLPE RETENTOR DE ENVELOPE
17    8/30/2016    2045  GOLPE RETENTOR DE ENVELOPE
18    8/30/2016    1087  GOLPE RETENTOR DE ENVELOPE
19    8/29/2016    3421         LEITORAS OBSTRUIDAS
20    8/29/2016    3426         LEITORAS OBSTRUIDAS
21    8/29/2016     766  GOLPE RETENTOR DE ENVELOPE
22    8/29/2016    3982                            
23    8/28/2016    4550    GOLPE RETENTOR DE CARTAO
24    8/28/2016    2083  GOLPE RETENTOR DE ENVELOPE
25    8/28/2016     572    GOLPE RETENTOR DE CARTAO
26    8/28/2016    4664  GOLPE RETENTOR DE ENVELOPE
27    8/28/2016    4265  GOLPE RETENTOR DE ENVELOPE
28    8/28/2016     657  GOLPE RETENTOR DE ENVELOPE
29    8/28/2016    1600  GOLPE RETENTOR DE ENVELOPE
..          ...     ...                         ...
872  03/05/2016    3982    GOLPE RETENTOR DE CARTAO
873  03/04/2016    3982    GOLPE RETENTOR DE CARTAO
874  03/03/2016    3982    GOLPE RETENTOR DE CARTAO
875  03/02/2016    3982    GOLPE RETENTOR DE CARTAO
876   7/31/2016    3989                         POC
877   7/30/2016    3989                         POC
878   7/29/2016    3989                         POC
879  06/04/2016    4025                         POC
880  06/03/2016    4025                         POC
881   4/14/2016    4028              TELEFONE FALSO
882   4/13/2016    4028              TELEFONE FALSO
883  05/05/2016    4056         LEITORAS OBSTRUIDAS
884  05/04/2016    4056         LEITORAS OBSTRUIDAS
885   5/30/2016    4100         LEITORAS OBSTRUIDAS
886   5/29/2016    4100         LEITORAS OBSTRUIDAS
887  03/06/2016    4181                         POC
888  03/05/2016    4181                         POC
889  03/04/2016    4181                         POC
890  03/02/2016    4353  GOLPE RETENTOR DE ENVELOPE
891  03/01/2016    4353  GOLPE RETENTOR DE ENVELOPE
892   7/19/2016    4395         LEITORAS OBSTRUIDAS
893   7/18/2016    4395         LEITORAS OBSTRUIDAS
894   7/17/2016    4395         LEITORAS OBSTRUIDAS
895   7/16/2016    4395         LEITORAS OBSTRUIDAS
896   7/15/2016    4395         LEITORAS OBSTRUIDAS
897   7/14/2016    4395         LEITORAS OBSTRUIDAS
898   7/13/2016    4395         LEITORAS OBSTRUIDAS
899  07/12/2016    4395         LEITORAS OBSTRUIDAS
900  04/12/2016    4664         LEITORAS OBSTRUIDAS
901  04/11/2016    4664         LEITORAS OBSTRUIDAS

[902 rows x 3 columns]

In [36]:
df_poc.printSchema()

root
 |-- data_range: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



In [37]:
df_poc = df_poc.withColumn('data_range', funcStrToDate_dmyyyy(col('data_range'))) 
df_poc.printSchema()
df_poc.toPandas()

root
 |-- data_range: date (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



data_range agencia                       golpe
0    2016-09-05    1546  GOLPE RETENTOR DE ENVELOPE
1    2016-09-05    1650         LEITORAS OBSTRUIDAS
2    2016-09-05     734         LEITORAS OBSTRUIDAS
3    2016-09-05    1546  GOLPE RETENTOR DE ENVELOPE
4    2016-09-05    4348  GOLPE RETENTOR DE ENVELOPE
5    2016-09-04     778              TELEFONE FALSO
6    2016-09-04     350  GOLPE RETENTOR DE ENVELOPE
7    2016-09-04    1206  GOLPE RETENTOR DE ENVELOPE
8    2016-09-04    1206  GOLPE RETENTOR DE ENVELOPE
9    2016-09-03     985  GOLPE RETENTOR DE ENVELOPE
10   2016-09-03     564  GOLPE RETENTOR DE ENVELOPE
11   2016-09-03    3793  GOLPE RETENTOR DE ENVELOPE
12   2016-09-03     768  GOLPE RETENTOR DE ENVELOPE
13   2016-09-03    4442    GOLPE RETENTOR DE CARTAO
14   2016-09-03     921  GOLPE RETENTOR DE ENVELOPE
15   2016-09-03     919  GOLPE RETENTOR DE ENVELOPE
16   2016-08-30    1566  GOLPE RETENTOR DE ENVELOPE
17   2016-08-30    2045  GOLPE RETENTOR DE ENVELOPE
18   2016-08-30    1087  GOLPE RETENTOR DE ENVELOPE
19   2016-08-29    3421         LEITORAS OBSTRUIDAS
20   2016-08-29    3426         LEITORAS OBSTRUIDAS
21   2016-08-29     766  GOLPE RETENTOR DE ENVELOPE
22   2016-08-29    3982                            
23   2016-08-28    4550    GOLPE RETENTOR DE CARTAO
24   2016-08-28    2083  GOLPE RETENTOR DE ENVELOPE
25   2016-08-28     572    GOLPE RETENTOR DE CARTAO
26   2016-08-28    4664  GOLPE RETENTOR DE ENVELOPE
27   2016-08-28    4265  GOLPE RETENTOR DE ENVELOPE
28   2016-08-28     657  GOLPE RETENTOR DE ENVELOPE
29   2016-08-28    1600  GOLPE RETENTOR DE ENVELOPE
..          ...     ...                         ...
872  2016-03-05    3982    GOLPE RETENTOR DE CARTAO
873  2016-03-04    3982    GOLPE RETENTOR DE CARTAO
874  2016-03-03    3982    GOLPE RETENTOR DE CARTAO
875  2016-03-02    3982    GOLPE RETENTOR DE CARTAO
876  2016-07-31    3989                         POC
877  2016-07-30    3989                         POC
878  2016-07-29    3989                         POC
879  2016-06-04    4025                         POC
880  2016-06-03    4025                         POC
881  2016-04-14    4028              TELEFONE FALSO
882  2016-04-13    4028              TELEFONE FALSO
883  2016-05-05    4056         LEITORAS OBSTRUIDAS
884  2016-05-04    4056         LEITORAS OBSTRUIDAS
885  2016-05-30    4100         LEITORAS OBSTRUIDAS
886  2016-05-29    4100         LEITORAS OBSTRUIDAS
887  2016-03-06    4181                         POC
888  2016-03-05    4181                         POC
889  2016-03-04    4181                         POC
890  2016-03-02    4353  GOLPE RETENTOR DE ENVELOPE
891  2016-03-01    4353  GOLPE RETENTOR DE ENVELOPE
892  2016-07-19    4395         LEITORAS OBSTRUIDAS
893  2016-07-18    4395         LEITORAS OBSTRUIDAS
894  2016-07-17    4395         LEITORAS OBSTRUIDAS
895  2016-07-16    4395         LEITORAS OBSTRUIDAS
896  2016-07-15    4395         LEITORAS OBSTRUIDAS
897  2016-07-14    4395         LEITORAS OBSTRUIDAS
898  2016-07-13    4395         LEITORAS OBSTRUIDAS
899  2016-07-12    4395         LEITORAS OBSTRUIDAS
900  2016-04-12    4664         LEITORAS OBSTRUIDAS
901  2016-04-11    4664         LEITORAS OBSTRUIDAS

[902 rows x 3 columns]

In [38]:
df_poc = df_poc \
    .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
    .withColumn('data_range', df_poc["data_range"].cast(TimestampType())) \
    .withColumn('golpe', df_poc["golpe"].cast(StringType()))
df_poc.printSchema()

root
 |-- data_range: timestamp (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



In [39]:
df_poc.limit(10).toPandas()

data_range agencia                       golpe
0 2016-05-08    1793              TELEFONE FALSO
1 2016-05-08    2200  GOLPE RETENTOR DE ENVELOPE
2 2016-05-07    2194  GOLPE RETENTOR DE ENVELOPE
3 2016-05-07     564  GOLPE RETENTOR DE ENVELOPE
4 2016-05-07     997  GOLPE RETENTOR DE ENVELOPE
5 2016-05-07    3809  GOLPE RETENTOR DE ENVELOPE
6 2016-05-07    2175  GOLPE RETENTOR DE ENVELOPE
7 2016-05-07     641  GOLPE RETENTOR DE ENVELOPE
8 2016-05-07     803    GOLPE RETENTOR DE CARTAO
9 2016-05-07    1549  GOLPE RETENTOR DE ENVELOPE

In [40]:
# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
df_poc = df_poc.where("CAST(data_range AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql_90,dt_sql_2)).orderBy("data_range")
#df_eventlog = df_eventlog.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
df_poc.limit(10).toPandas()

data_range agencia                       golpe
0 2016-05-10    4424              TELEFONE FALSO
1 2016-05-10    4425         LEITORAS OBSTRUIDAS
2 2016-05-10     114  GOLPE RETENTOR DE ENVELOPE
3 2016-05-10    3733         LEITORAS OBSTRUIDAS
4 2016-05-10    4279  GOLPE RETENTOR DE ENVELOPE
5 2016-05-11     114  GOLPE RETENTOR DE ENVELOPE
6 2016-05-12     909  GOLPE RETENTOR DE ENVELOPE
7 2016-05-13      86                         POC
8 2016-05-13    4433                            
9 2016-05-13    4433         LEITORAS OBSTRUIDAS

In [41]:
df_poc.printSchema()

root
 |-- data_range: timestamp (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



In [42]:
#    .map(lambda p: (p[4],p[2],p[5], p[9])) \
#df_poc = df_poc.withColumn('data_cad', funcStrToDate_dmyyyy(col('data_cad'))) 

#df_poc = df_poc \
#   .withColumn('atm', df_poc["atm"].cast(StringType()))\
#   .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
##    .withColumn('data_cad', df_poc["data_cad"].cast(TimestampType())) \
#   .withColumn('golpe', df_poc["golpe"].cast(StringType())) \

In [44]:
# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
#df_poc = df_poc.where("CAST(data_cad AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql_90,dt_sql_2)).orderBy("data_cad")
#df_eventlog = df_eventlog.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
#df_poc.limit(10).toPandas()

IndentationError: unexpected indent (<ipython-input-44-2df8425f837f>, line 3)

In [45]:
#rdd_poc.collect()

In [47]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_poc = df_poc\
    .withColumn('agencia', trim(df_poc.agencia))\
    .withColumn('agencia', regexp_replace('agencia', '^0+(?!$)', '')) # removendo zero `esquerda
#df_poc.limit(10).toPandas()

In [48]:
print ("\nNúmero de linhas carregadas: " + str("%d" % (df_poc.count())))
print ("\nNúmero de linhas após retirar NAs: " + str("%d" % (df_poc.dropna().count())))


Número de linhas carregadas: 447

Número de linhas após retirar NAs: 447


## Preparando dados processados
Marcando como POC as entradas do log para os eventos marcadores 

In [49]:
# APROXIMACAO ABAIXO PARA EVITAR PROBLEMAS NO withColumns (TIRAR SE NAO USAR PIVOT DE PANDAS)
# criar coluna de indice de proximidade da mediana (iproxmed)
# calculado como número de ocorrências real diaria/mediana diaria de eventos por agencia e atm
# If you use deimcal, you need to import

#from decimal import getcontext, Decimal
# Set the precision.
#getcontext().prec = 3

#format(math.pi, '.2f')
# mudar o arredondamento abaixo para ver se afeta o modelo
df_feat = df_eventlog2\
    .withColumn('iproxmed', (col("nr_eventos")/col("mediana"))*1000)
df_feat.limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos  mediana  \
0          105     158         1653 2016-06-23           1      1.0   
1          105     158         1653 2016-05-28           1      1.0   
2          105     158         1653 2016-06-03           1      1.0   
3          105     158         1653 2016-06-15           1      1.0   
4          105     158         1653 2016-07-25           1      1.0   
5          105     158         1653 2016-07-20           1      1.0   
6          105     158         1653 2016-06-09           1      1.0   
7          105     158         1653 2016-07-06           1      1.0   
8          105     158         1653 2016-08-08           1      1.0   
9          105     158         1653 2016-05-11           1      1.0   

   percentil_max  iproxmed  
0            1.0    1000.0  
1            1.0    1000.0  
2            1.0    1000.0  
3            1.0    1000.0  
4            1.0    1000.0  
5            1.0    1000.0  
6            1.0    1000.0  
7            1.0    1000.0  
8            1.0    1000.0  
9            1.0    1000.0

In [50]:
# transpondo tipo de evento para gerar features
df_feat = df_feat\
    .drop(df_feat["mediana"])

In [51]:
# voltar a usar aqui o pandas para transposicao pois erro no sqlcontext
pd_pivot = df_feat\
    .select('dh_even', 'nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'iproxmed').toPandas()

In [52]:
pd_feat = pd.pivot_table(pd_pivot,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()

pd_feat.columns = [c[1] if c[1] else c[0] for c in pd_feat.columns.tolist()]
pd_feat = pd_feat.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
pd_feat

nr_idef_secu cd_eqpt    1599  1642  1648    1650  1651  1653  1656  \
0              105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
1              105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
2              105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
3              105   10001  1000.0   0.0   0.0     0.0   0.0   0.0   0.0   
4              105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
5              105   10001  2000.0   0.0   0.0     0.0   0.0   0.0   0.0   
6              105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
7              105   10001  1000.0   0.0   0.0     0.0   0.0   0.0   0.0   
8              105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
9              105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
10             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
11             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
12             105   10001  1000.0   0.0   0.0     0.0   0.0   0.0   0.0   
13             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
14             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
15             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
16             105   10001  1000.0   0.0   0.0     0.0   0.0   0.0   0.0   
17             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
18             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
19             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
20             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
21             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
22             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
23             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
24             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
25             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
26             105   10001  1000.0   0.0   0.0     0.0   0.0   0.0   0.0   
27             105   10001  1000.0   0.0   0.0     0.0   0.0   0.0   0.0   
28             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
29             105   10001     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
...            ...     ...     ...   ...   ...     ...   ...   ...   ...   
90688          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90689          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90690          990     153  1000.0   0.0   0.0     0.0   0.0   0.0   0.0   
90691          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90692          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90693          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90694          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90695          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90696          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90697          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90698          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90699          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90700          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90701          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90702          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90703          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90704          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90705          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90706          990     153     0.0   0.0   0.0  1000.0   0.0   0.0   0.0   
90707          990     153     0.0   0.0   0.0     0.0   0.0   0.0   0.0   
90708          990     153     0.0   0.0   0.0     0.0

In [53]:
pd_feat.columns

Index([u'nr_idef_secu',      u'cd_eqpt',         u'1599',         u'1642',
               u'1648',         u'1650',         u'1651',         u'1653',
               u'1656',         u'1657',         u'1658',         u'1765',
               u'1769',         u'1813',         u'2034',         u'2267',
               u'2422',         u'2430',         u'2492',         u'2493',
               u'2761',         u'2767',         u'2982',         u'2996',
               u'3043',         u'3260',         u'3321',         u'3425',
               u'3812',         u'3814',         u'4266',         u'4267',
               u'4268',         u'4269',         u'4274',         u'5104',
               u'5278',         u'5495',         u'5498',          u'569',
                u'676',         u'6776',         u'6778',          u'679',
                u'682',          u'684',          u'723',          u'724',
                u'791',          u'798',          u'799',          u'899',
          u'data_even'],


In [84]:
df_feat_alt = sqlCtx.createDataFrame(pd_feat)

In [85]:
#df_feat_alt.limit(10).toPandas()

In [86]:
# testar abaixo se este mesmo evento se encontra no seletor de poc e porque nao aparece aqui
#df_feat.filter(col('nr_sequ_even') == "1746").count()

In [87]:
# na verdade nao precisa executar o codigo abaixo na query de transposicao
# criando lista somente com pokeventos que nao existem no log
# temporariamente carregando l_evenpoc somente com poucos eventos por falta de memoria
# ver em producao se roda
#l_evenpoc = ["5401","1746","666","1903","2034","2930","4236","6732", "1588"]
#existe = [even if df_feat.filter(col('nr_sequ_even') == even).count() == 0 else None for even in l_evenpoc]
#existe

# Provavel Bug no Pyspark impede abordagem abaixo para nao usar  pivot do Pandas (ao corrigir, substituir)

#df_poc.registerTempTable("poc")

#df_feat = sqlContext.sql("select nr_idef_secu,cd_eqpt,dh_even,\
#                        max(ev4266) as ev4266,\
#                        max(ev5490) as ev5490,\
#                        max(ev899) as ev899,\
#                        max(ev2034) as ev2034,\
#                        max(ev4267) as ev4267,\
#                        max(ev4268) as ev4268,\
#                        max(ev2982) as ev2982,\
#                        max(ev3260) as ev3260\
#                    from (select nr_idef_secu,cd_eqpt,dh_even,\
#                                  case when nr_sequ_even='4266' then iproxmed else 0 end ev4266,\
#                                  case when nr_sequ_even='5490' then iproxmed else 0 end ev5490,\
#                                  case when nr_sequ_even='899' then iproxmed else 0 end ev899,\
#                                  case when nr_sequ_even='2034' then iproxmed else 0 end ev2034,\
#                                  case when nr_sequ_even='4267' then iproxmed else 0 end ev4267,\
#                                  case when nr_sequ_even='4268' then iproxmed else 0 end ev4268,\
#                                  case when nr_sequ_even='2982' then iproxmed else 0 end ev2982,\
#                                  case when nr_sequ_even='3260' then iproxmed else 0 end ev3260 \
#                             from df_feat) x \
#                     group by nr_idef_secu,cd_eqpt,dh_even")

#df_feat_alt.limit(10).toPandas()

#l_features = ['ev4266', 'ev5490', 'ev899', 'ev2034', 'ev4267', 'ev4268', 'ev2982', 'ev3260']

In [88]:
#pd_feat.columns = [c[1] if c[1] else c[0] for c in pd_feat.columns.tolist()]

In [89]:
#pd_feat

In [90]:
#c = pd_feat.columns.tolist
#c[1]

In [91]:
# criar coluna com data como string para posterior transformação para dataframe
#pd_feat = pd_feat.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
#pd_feat

In [92]:
# salva como csv pois conversao para Dataframe Spark apresenta erro. provavel falta de memoria
#pd_feat.to_csv(tabelao_out)

In [93]:
#pd_feat.columns

In [94]:
#df_feat_alt = sqlCtx.createDataFrame(pd_feat)
# FALTA converter data em timedate de novo (converter para string no Pandas antes)

In [95]:
#df_feat_alt.take(5)

In [96]:
#df_feat_alt.limit(10).toPandas()

In [97]:
# OBS: se já tiver o tabelao_alt gerado, pode rodar a partir daqui!!!!!
#############################################################################
# AQUI TIVE QUE FAZER ESTA GAMBIARRA: salvar como csv e alguma manipulaçao (acertar cabeçalho) para depois ler
# até resolver o problema de memoria gerado por fazer o pivot no Pandas
#rdd_feat_alt = sc.textFile(tabelao_in)

In [98]:
#rdd_feat_alt.first()
#df = sqlContext.load(source="com.databricks.spark.csv", header="true", path = "cars.csv")

In [99]:
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_feat_alt.count())))

In [100]:
#header = rdd_feat_alt.first()
#schemaString = header
#fields = [StructField(field_name, StringType(), False) for field_name in schemaString.split(',')]
#schema = StructType(fields)

#rdd_feat_alt = rdd_feat_alt \
##    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(","))

#df_feat_alt = sqlContext.createDataFrame(rdd_feat_alt, schema)

In [101]:
#df_feat_alt.limit(10).toPandas()

In [102]:
#df_feat_alt.printSchema()

In [103]:
df_feat_alt = df_feat_alt.withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))
#df_feat_alt.limit(10).toPandas()

In [104]:
df_feat_alt.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 1599: double (nullable = true)
 |-- 1642: double (nullable = true)
 |-- 1648: double (nullable = true)
 |-- 1650: double (nullable = true)
 |-- 1651: double (nullable = true)
 |-- 1653: double (nullable = true)
 |-- 1656: double (nullable = true)
 |-- 1657: double (nullable = true)
 |-- 1658: double (nullable = true)
 |-- 1765: double (nullable = true)
 |-- 1769: double (nullable = true)
 |-- 1813: double (nullable = true)
 |-- 2034: double (nullable = true)
 |-- 2267: double (nullable = true)
 |-- 2422: double (nullable = true)
 |-- 2430: double (nullable = true)
 |-- 2492: double (nullable = true)
 |-- 2493: double (nullable = true)
 |-- 2761: double (nullable = true)
 |-- 2767: double (nullable = true)
 |-- 2982: double (nullable = true)
 |-- 2996: long (nullable = true)
 |-- 3043: double (nullable = true)
 |-- 3260: long (nullable = true)
 |-- 3321: double (nullable = true)
 |-- 3425: doubl

In [105]:
df_feat_alt = df_feat_alt\
    .withColumn('nr_idef_secu', trim(df_feat_alt.nr_idef_secu))
#df_feat_alt.printSchema()

In [106]:
df_feat_alt.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 1599: double (nullable = true)
 |-- 1642: double (nullable = true)
 |-- 1648: double (nullable = true)
 |-- 1650: double (nullable = true)
 |-- 1651: double (nullable = true)
 |-- 1653: double (nullable = true)
 |-- 1656: double (nullable = true)
 |-- 1657: double (nullable = true)
 |-- 1658: double (nullable = true)
 |-- 1765: double (nullable = true)
 |-- 1769: double (nullable = true)
 |-- 1813: double (nullable = true)
 |-- 2034: double (nullable = true)
 |-- 2267: double (nullable = true)
 |-- 2422: double (nullable = true)
 |-- 2430: double (nullable = true)
 |-- 2492: double (nullable = true)
 |-- 2493: double (nullable = true)
 |-- 2761: double (nullable = true)
 |-- 2767: double (nullable = true)
 |-- 2982: double (nullable = true)
 |-- 2996: long (nullable = true)
 |-- 3043: double (nullable = true)
 |-- 3260: long (nullable = true)
 |-- 3321: double (nullable = true)
 |-- 3425: doubl

In [107]:
# primeiro left_join df_feat_alt + df_poc
    # depois para agencia == cd_secu and cada coluna evento > 1 = POC
df_tidy = df_feat_alt \
    .join(df_poc, (df_feat_alt['nr_idef_secu'] == df_poc['agencia']) &
            (df_feat_alt['data_even'] == df_poc['data_range']), 'left')

In [108]:
# marca label de ocorrencia de POC
df_tidy = df_tidy.withColumn('label',
            when(col("agencia") == col("nr_idef_secu"), 1.0)\
            .otherwise(0.0))\
            .drop(df_tidy["agencia"]).drop(df_tidy["data_range"])
                            
#df_tidy.limit(10).toPandas()

In [109]:
#df_tidy.orderBy(col('agencia').desc())\
#    .filter(col('agencia') == col('nr_idef_secu'))\
#    .select('agencia').distinct().toPandas()
# 53 agências distintas com POC

In [110]:
df_tidy.limit(10).toPandas()

nr_idef_secu cd_eqpt  1599    1642    1648    1650  1651    1653  1656  \
0          105   10001   0.0     0.0     0.0     0.0   0.0     0.0   0.0   
1          105     151   0.0     0.0     0.0     0.0   0.0     0.0   0.0   
2          105     152   0.0     0.0     0.0     0.0   0.0     0.0   0.0   
3          105     153   0.0     0.0     0.0  1000.0   0.0  1000.0   0.0   
4          105     154   0.0     0.0     0.0     0.0   0.0     0.0   0.0   
5          105     156   0.0  1000.0     0.0     0.0   0.0     0.0   0.0   
6          105     158   0.0     0.0     0.0     0.0   0.0     0.0   0.0   
7          105     159   0.0     0.0     0.0     0.0   0.0     0.0   0.0   
8          105     160   0.0     0.0     0.0     0.0   0.0     0.0   0.0   
9          105     161   0.0     0.0  1000.0  1000.0   0.0  1000.0   0.0   

   1657  ...       684   723     724  791  798     799  899  data_even  golpe  \
0   0.0  ...       0.0     0     0.0  0.0  0.0     0.0  0.0 2016-08-01    NaN   
1   0.0  ...       0.0  1000   500.0  0.0  0.0     0.0  0.0 2016-08-01    NaN   
2   0.0  ...       0.0  1000  1000.0  0.0  0.0  1000.0  0.0 2016-08-01    NaN   
3   0.0  ...       0.0     0  1000.0  0.0  0.0     0.0  0.0 2016-08-01    NaN   
4   0.0  ...       0.0     0  1000.0  0.0  0.0  1000.0  0.0 2016-08-01    NaN   
5   0.0  ...    1000.0  1000  1500.0  0.0  0.0     0.0  0.0 2016-08-01    NaN   
6   0.0  ...       0.0  1000   500.0  0.0  0.0     0.0  0.0 2016-08-01    NaN   
7   0.0  ...       0.0  1000   500.0  0.0  0.0     0.0  0.0 2016-08-01    NaN   
8   0.0  ...       0.0  1000  1000.0  0.0  0.0     0.0  0.0 2016-08-01    NaN   
9   0.0  ...       0.0  1000  1500.0  0.0  0.0     0.0  0.0 2016-08-01    NaN   

   label  
0    0.0  
1    0.0  
2    0.0  
3    0.0  
4    0.0  
5    0.0  
6    0.0  
7    0.0  
8    0.0  
9    0.0  

[10 rows x 55 columns]

In [111]:
l_primeven = l_features[0]
df_tidy_dbl = df_tidy\
       .withColumn(l_primeven, df_tidy[l_primeven].cast(DoubleType()))\

for even in l_features[1:]:
    df_tidy_dbl = df_tidy_dbl.withColumn(even, df_tidy_dbl[even].cast(DoubleType()))
#df_tidy_dbl = df_tidy.withColumn('2492', df_tidy['2492'].cast(DoubleType()))\
#        .withColumn('2996', df_tidy['2996'].cast(DoubleType()))\
#        .withColumn('3217', df_tidy['3217'].cast(DoubleType()))\
#        .withColumn('3260', df_tidy['3260'].cast(DoubleType()))\
#        .withColumn('3551', df_tidy['3551'].cast(DoubleType()))\
#        .withColumn('3552', df_tidy['3552'].cast(DoubleType()))\
#        .withColumn('3554', df_tidy['3554'].cast(DoubleType()))\
#        .withColumn('3829', df_tidy['3829'].cast(DoubleType()))\
#        .withColumn('4609', df_tidy['4609'].cast(DoubleType()))
#        .withColumn('6778', df_tidy['6778'].cast(DoubleType()))
#df_tidy_dbl = df_tidy

In [112]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 1599: double (nullable = true)
 |-- 1642: double (nullable = true)
 |-- 1648: double (nullable = true)
 |-- 1650: double (nullable = true)
 |-- 1651: double (nullable = true)
 |-- 1653: double (nullable = true)
 |-- 1656: double (nullable = true)
 |-- 1657: double (nullable = true)
 |-- 1658: double (nullable = true)
 |-- 1765: double (nullable = true)
 |-- 1769: double (nullable = true)
 |-- 1813: double (nullable = true)
 |-- 2034: double (nullable = true)
 |-- 2267: double (nullable = true)
 |-- 2422: double (nullable = true)
 |-- 2430: double (nullable = true)
 |-- 2492: double (nullable = true)
 |-- 2493: double (nullable = true)
 |-- 2761: double (nullable = true)
 |-- 2767: double (nullable = true)
 |-- 2982: double (nullable = true)
 |-- 2996: double (nullable = true)
 |-- 3043: double (nullable = true)
 |-- 3260: double (nullable = true)
 |-- 3321: double (nullable = true)
 |-- 3425: d

In [113]:
df_tidy_dbl.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|89483|
|  1.0| 1273|
+-----+-----+



In [114]:
# tabelao com labels
#df_tidy.toPandas().to_csv(tabelao_label)

In [115]:
#df_tidy.printSchema()

In [116]:
# ATANCAO: ver se falta fazer antes
#df_tidy = df_feat_alt\
#   .withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))\
#    .withColumn('label', df_feat_alt["label"].cast(DoubleType()))

In [117]:
#df_tidy.limit(10).toPandas()

In [118]:
#df_tidy.printSchema()

In [119]:
# salvando tabelao com label
#df_tidy.toPandas().to_csv(tabelao_final)

## Modelagem
O modelo usado será o de Random Forest, implementando pipelines da library ML do Spark.
A partir da amostra de dados será selecionado um subset com 75% dos dados para treinar o modelo. 

### Treino do Modelo
Prepara a amostra de acordo com o formato de entrada de label e vetor de features esperados pelo modelo.

In [120]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [121]:
#df_tidy_dbl.printSchema()

In [122]:
# rodando para as features com mais relevância já analisadas
#assemblerTrain = VectorAssembler(
#    inputCols=["1641", "1642", "2034", "2982", "3260", "3425", "4266", "4268", "4269", "899"],
#   outputCol="features")
assemblerTrain = VectorAssembler(
       inputCols=l_features,
       outputCol="features")
#    inputCols=["1641", "1642", "2034", "2982", "3260", "3425", "4266", "4268", "4269", "899"],
#               ,"1617", "1689","2433", "1746", "1568","883", "2163"],
 

In [123]:
df_tmp = assemblerTrain.transform(df_tidy_dbl)

In [124]:
df_tidy_rf = df_tmp.select("nr_idef_secu", "cd_eqpt", "label", "features")
df_tidy_rf.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [125]:
df_tidy_rf.take(5)

[Row(nr_idef_secu=u'112', cd_eqpt=u'10001', label=0.0, features=SparseVector(50, {28: 1000.0})),
 Row(nr_idef_secu=u'112', cd_eqpt=u'151', label=0.0, features=SparseVector(50, {2: 500.0, 5: 1000.0, 7: 2000.0, 17: 1000.0, 19: 500.0, 28: 1000.0, 49: 2000.0})),
 Row(nr_idef_secu=u'112', cd_eqpt=u'152', label=0.0, features=SparseVector(50, {2: 500.0, 18: 1000.0, 19: 1000.0, 28: 2000.0, 31: 1000.0})),
 Row(nr_idef_secu=u'112', cd_eqpt=u'153', label=0.0, features=SparseVector(50, {2: 3000.0, 7: 1000.0, 18: 1000.0, 19: 500.0, 28: 1000.0, 45: 1000.0})),
 Row(nr_idef_secu=u'112', cd_eqpt=u'154', label=0.0, features=SparseVector(50, {19: 500.0, 28: 1000.0}))]

In [126]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_rf)
#labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(trainingData_rf)

In [127]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 2 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_rf)
#featureIndexer =\
#    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(trainingData_rf)

In [128]:
# Split the data into training and test sets (30% held out for testing)
(trainingData_rf, testData_rf) = df_tidy_rf.randomSplit([0.7, 0.3])

In [129]:
trainingData_rf.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|62488|
|  1.0|  899|
+-----+-----+



In [130]:
testData_rf.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|26995|
|  1.0|  374|
+-----+-----+



Separa a amostra de forma aleatória em dados de treino (70% da amostra) e de teste (30% da amostra, para posterior avaliação do modelo).

In [131]:
# Train a Random Forest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
#dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [132]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [133]:
model = pipeline.fit(trainingData_rf)

In [134]:
#pipeline = Pipeline(stages=[rf])
# Search through decision tree's maxDepth parameter for best model
#from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
#paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,3,4,5,6,7]).build()

#numFolds = 5

In [135]:
#crossval = CrossValidator(
#    estimator=pipeline,
#    estimatorParamMaps=paramGrid,
#    evaluator=evaluator,
#    numFolds=numFolds)

In [136]:
# Fetch best model
#tree_model = model.bestModel.stages[2]
#print tree_model

In [137]:
# Train model.  This also runs the indexers.
#model = pipeline.fit(trainingData_rf)

## Persistência do Modelo
Salva o modelo para posterior uso. O código comentado abaixo deve ser descomentado nas manutenções para testar se modelo foi corretamente salvo.

In [138]:
# SAVE MODEL FOR FUTURE USE (TEST IF SAVED OK)
#model.save(sc, savedModel) # Erro. Funciona com mllib. Pesquisar como salvar pipeline
# persist model to HDFS
#model.save(sc, tabelao_out)
#sc.parallelize(Seq(model), 1).saveAsObjectFile("hdfs:///user/root/linReg.model")

# READ MODEL
#linRegModel = sc.objectFile[LinearRegressionModel]("linReg.model").first()

### Avaliação do Modelo
Avalia o modelo usando os dados separados para teste, obtendo o erro de teste (1 - acurácia), a área sob a curva ROC (AUC) e a taxa Precision/Recall.

In [139]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.transform(testData_rf)

In [140]:
evaluator = BinaryClassificationEvaluator()

In [141]:
evaluator.explainParams()

'labelCol: label column name (default: label)\nmetricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)'

In [142]:
auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

In [143]:
print("AUC = %g " % (auroc))

AUC = 0.578672 


In [144]:
precrec = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})

In [145]:
print("Precision/Recall = %g " % (precrec))

Precision/Recall = 0.0212056 


In [146]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.423059 


In [147]:
#predictions.select

Abaixo são mostradas as primeiras linhas da tabela de previsão gerada pelo modelo

In [148]:
predictions.select("nr_idef_secu", "cd_eqpt", "prediction", "indexedLabel", "features", "probability").take(5)

[Row(nr_idef_secu=u'112', cd_eqpt=u'10001', prediction=0.0, indexedLabel=0.0, features=SparseVector(50, {28: 1000.0}), probability=DenseVector([0.9858, 0.0142])),
 Row(nr_idef_secu=u'117', cd_eqpt=u'168', prediction=0.0, indexedLabel=0.0, features=SparseVector(50, {}), probability=DenseVector([0.9858, 0.0142])),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'10001', prediction=0.0, indexedLabel=1.0, features=SparseVector(50, {6: 1000.0}), probability=DenseVector([0.985, 0.015])),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'154', prediction=0.0, indexedLabel=1.0, features=SparseVector(50, {28: 1000.0}), probability=DenseVector([0.9858, 0.0142])),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'155', prediction=0.0, indexedLabel=1.0, features=SparseVector(50, {7: 1000.0, 11: 1000.0, 16: 1000.0, 19: 1000.0, 29: 500.0, 31: 750.0}), probability=DenseVector([0.9869, 0.0131]))]

In [114]:
#x = predictions.map(lambda a: (a[0], a[1], a[7][1]))

In [149]:
#from pyspark.mllib.linalg import Vectors
#def extract(row):
#    return (row.nr_idef_secu, row.cd_eqpt, ) + tuple(float(x) for x in row.probability.values)

#x = predictions.select("nr_idef_secu", "cd_eqpt", "probability")

In [150]:
#x.show()

+------------+-------+--------------------+
|nr_idef_secu|cd_eqpt|         probability|
+------------+-------+--------------------+
|        1053|    153|[0.98713437424171...|
|        1053|  10001|[0.98503121606987...|
|        1207|    151|[0.98581261928025...|
|        1523|  10001|[0.98581261928025...|
|        1523|    151|[0.98581261928025...|
|        1523|    160|[0.98586106320924...|
|        1523|    168|[0.98663773091026...|
|         177|  10001|[0.98503121606987...|
|         177|    151|[0.98720708692873...|
|         177|    155|[0.98541655147998...|
|         177|    156|[0.98534694335599...|
|         177|    158|[0.98701616271439...|
|        2014|    152|[0.98684391768799...|
|        2014|    151|[0.98626003023641...|
|        2014|    153|[0.98701616271439...|
|        2014|    157|[0.98622783739978...|
|        2093|    152|[0.98581261928025...|
|        2093|    155|[0.98586106320924...|
|        2093|    156|[0.98663773091026...|
|        2093|    160|[0.9858126

In [152]:
#x.rdd.map(extract).toDF(["nr_idef_secu", "cd_eqpt", "probFALSE", "probTRUE" ])

In [153]:
#x.first()

In [154]:
#x.toDF(['agencia',
#          'atm', 
#           'ranking'])

In [155]:
#df_x = sqlCtx.createDataFrame(x)

In [156]:
#df_x

In [180]:
# 6. dividir dados em treino e teste
# selecionando sample estratificada
#df_tidy_dbl.groupBy("label").count().orderBy("label").show()
# N: 0.9995   S: 0.000499
# Balanceamento com marcação anterior de report de POC
# N: 0.0110104 S: 0.98898
# Balanceamento com marcação por evento fora do percentil de POC
# N:  S: 

In [181]:
#df_tidy_dbl.groupBy(["nr_idef_secu", "label"]).count().orderBy("nr_idef_secu").show()

In [182]:
# salving the resulting dataframe
#type(poc_predicted)
#poc_predicted.to_csv(previsao)
#df.write.csv('mycsv.csv')

In [183]:
# estratégia
# 1. obter a média diária da agência + atm (trocar para mediana) para cada evento
# 2. do log, somar o número de ocorrências por agencia + atm + evento agrupado por dia
# 3. montar tabelao inicial e criar coluna count/media para ver índice de desvio da média (FEATURE) por agencia/atm
# 4. criar coluna LABEL concatenando com arquivo POC por agencia + data cad (atm nao confiavel no arquivo)
# obs: desvio > 1 indica percentual acima da média e vice-versa
# 5. fazer o pivot como acima

# trocar media por mediana (FALTA)
# usar mediana para gerar o tabelao (ou percentil 75% ou soma contante) (FALTA)
# dividir pelo count para ter a feature percentil/count (FALTA)
# transposicao (filtrar tabelao por agencia + atm e join dois a dois (em loop?))
# filtrar os 3 dias após a data de poc para tirar efeitos de setup   
# 

In [184]:
# matriz de confusão
#poc_predicted

## confusion matrix

In [157]:
def confusion_matrix(prevCol, labelCol):
    if prevCol == 1.0 and labelCol == 1.0:
        return 'TP'
    else:
        if prevCol == 1.0 and labelCol == 0.0:
            return 'FP'
        else:
            if prevCol == 0.0 and labelCol == 0.0:
                return 'TN'
            else:
                if prevCol == 0.0 and labelCol == 1.0:
                    return 'FN'
            
createNewColFromTwo = udf(confusion_matrix, StringType())
df_conf_matr = predictions.withColumn('conf_matr', createNewColFromTwo(predictions['prediction'],
            predictions['label']))

In [158]:
# matriz de confuao
df_rates = df_conf_matr.groupBy("conf_matr").count()
df_rates.toPandas()

conf_matr  count
0        TN  26995
1        FN    374

In [143]:
# Sensibility - habilidade do modelo classificar corretamente um POC
# Probability of being test positive when POC present.
# TP / (TP + FN)
#sensitivity = float(df_rates.collect()[0][1]) / (df_rates.collect()[0][1] + df_rates.collect()[2][1])
# Specificity - habilidade do modelo classificar corretamente a ausência de POC 
#Probability of being test negative when POC absent.
# TN / (TN + FP)
#specificity = float(df_rates.collect()[1][1]) / (df_rates.collect()[1][1] + df_rates.collect()[3][1])
# precisao
# TP / (TP + FP)
#precisao = float(df_rates.collect()[0][1]) / (df_rates.collect()[0][1] + df_rates.collect()[3][1])

#print ("\nSentitividade ou Recall: " + str("%.2f" % (sensitivity)))
#print ("\nEspecificidade         : " + str("%.2f" % (specificity)))
#print ("\nPrecisão               : " + str("%.2f" % (precisao)))

IndexError: list index out of range

In [144]:
#df_rates.toPandas()

In [159]:
df_rates.collect()

[Row(conf_matr=u'TN', count=26995), Row(conf_matr=u'FN', count=374)]

In [160]:
# TESTES
predictions.filter((col("prediction") == 0.0) & (col("label") == 1.0)).count()

365

In [161]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 1.0)).count()

0

In [162]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 0.0)).count()

0

In [163]:
predictions.filter((col("prediction") == 0.0) & (col("label") == 0.0)).count()

26977

In [164]:
# checagem do marcador de POC contra POCs reportados
# somente POC retencao de cartão, D-90
# trazer número de POCs para a chave agencia
# encotradas 52 agências com POC
#df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
#df_p.toPandas()
# agora comparar com o arquivo gerado com label

In [137]:
# foram encontradas agencias de POC nos eventos abaixo filtrados
#df_l = df_feat.where(col('label') == 1.0).groupBy('nr_idef_secu').count().orderBy('nr_idef_secu')
#df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
#df_l.toPandas()

In [136]:
# checar se selecionou realmente somente agencias com POC do log geral
#df_evenpoc.groupBy('nr_idef_secu').count().orderBy('nr_idef_secu').toPandas()

In [134]:
# checar quantas agencias reportadas como POC cairam no log com nova marcação (deveriam aparecer todos )
#df_j = df_l \
#    .join(df_p, (df_l['nr_idef_secu'] == df_p['agencia']), 'inner')
    
    

In [135]:
#df_j.limit(10).toPandas()